In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# InsightSpike-AI: Maze 51x51 Full Run (Colab Pro)

This notebook runs a production-scale 51x51 maze experiment using the L3-only paper preset and saves results to Google Drive.

Notes:
- Runtime > Change runtime type > GPU (A100 if available). CPU also works.
- Results and figures are saved under Drive.


In [11]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


## 1. Setup Environment


In [12]:
import os

REPO_URL = 'https://github.com/miyauchikazuyoshi/InsightSpike-AI.git'
REPO_DIR = '/content/InsightSpike-AI'

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL}

%cd /content/InsightSpike-AI
!pip -q install -e .
!pip -q install networkx numpy matplotlib pandas


/content/InsightSpike-AI
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for insightspike-ai (pyproject.toml) ... done


## 2. Configure Run Parameters


In [13]:
import os
import time

DRIVE_ROOT = '/content/drive/MyDrive/insightspike/maze_51x51_full'
RUN_TAG = time.strftime('maze51x51_s1500_seeds13_%Y%m%d_%H%M%S')
RUN_DIR = os.path.join(DRIVE_ROOT, RUN_TAG)

MAZE_SIZE = 51
MAX_STEPS = 1500
SEEDS = 13
SEED_START = 0

SUMMARY_PATH = os.path.join(RUN_DIR, f'_{MAZE_SIZE}x{MAZE_SIZE}_s{MAX_STEPS}_l3only_summary.json')
STEPS_PATH = os.path.join(RUN_DIR, f'_{MAZE_SIZE}x{MAZE_SIZE}_s{MAX_STEPS}_l3only_steps.json')
SQLITE_PATH = os.path.join(RUN_DIR, f'mq_{MAZE_SIZE}x{MAZE_SIZE}.sqlite')
SNAPSHOT_JSON = os.path.join(RUN_DIR, f'maze_{MAZE_SIZE}x{MAZE_SIZE}.json')
FIG_DIR = os.path.join(RUN_DIR, 'figs')
LOG_DIR = os.path.join(RUN_DIR, 'logs')
MPL_DIR = os.path.join(RUN_DIR, 'mpl')

os.makedirs(RUN_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MPL_DIR, exist_ok=True)

print('RUN_DIR:', RUN_DIR)
print('SUMMARY_PATH:', SUMMARY_PATH)
print('STEPS_PATH:', STEPS_PATH)


RUN_DIR: /content/drive/MyDrive/insightspike/maze_51x51_full/maze51x51_s1500_seeds13_20251229_152707
SUMMARY_PATH: /content/drive/MyDrive/insightspike/maze_51x51_full/maze51x51_s1500_seeds13_20251229_152707/_51x51_s1500_l3only_summary.json
STEPS_PATH: /content/drive/MyDrive/insightspike/maze_51x51_full/maze51x51_s1500_seeds13_20251229_152707/_51x51_s1500_l3only_steps.json


## 3. Configure Environment Variables


In [14]:
import os

os.environ['INSIGHTSPIKE_PRESET'] = 'paper'
os.environ['INSIGHTSPIKE_LITE_MODE'] = '1'
os.environ['INSIGHTSPIKE_MIN_IMPORT'] = '1'
os.environ['INSIGHTSPIKE_LOG_DIR'] = LOG_DIR
os.environ['MPLCONFIGDIR'] = MPL_DIR


## 4. Run Experiment (L3-only)
This may take time at full scale. Results will be saved to Drive.


In [15]:
!python experiments/maze-query-hub-prototype/tools/run_paper_l3_only.py   --maze-size {MAZE_SIZE}   --max-steps {MAX_STEPS}   --seeds {SEEDS}   --seed-start {SEED_START}   --out-root "{RUN_DIR}"   --namespace "{RUN_TAG}"   --sqlite "{SQLITE_PATH}"   --maze-snapshot-out "{SNAPSHOT_JSON}


/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


## 5. Resolve Output Paths (if re-running cells)
If you re-run the config cell after a long experiment, RUN_DIR can change.
This cell finds the latest summary on Drive and rebinds paths.


In [ ]:
import glob
import os


def _find_latest_summary(root: str, maze_size: int, max_steps: int) -> str | None:
    patterns = [
        os.path.join(
            root,
            f"maze{maze_size}x{maze_size}_s{max_steps}_seeds*",
            f"_{maze_size}x{maze_size}_s{max_steps}_l3only_summary.json",
        ),
        os.path.join(
            root,
            "*",
            f"_{maze_size}x{maze_size}_s{max_steps}_l3only_summary.json",
        ),
    ]
    candidates = []
    for pattern in patterns:
        candidates.extend(glob.glob(pattern))
    if not candidates:
        return None
    candidates.sort(key=os.path.getmtime)
    return candidates[-1]


if not os.path.exists(SUMMARY_PATH):
    latest = _find_latest_summary(DRIVE_ROOT, MAZE_SIZE, MAX_STEPS)
    if latest:
        RUN_DIR = os.path.dirname(latest)
        SUMMARY_PATH = latest
        STEPS_PATH = os.path.join(RUN_DIR, f"_{MAZE_SIZE}x{MAZE_SIZE}_s{MAX_STEPS}_l3only_steps.json")
        SQLITE_PATH = os.path.join(RUN_DIR, f"mq_{MAZE_SIZE}x{MAZE_SIZE}.sqlite")
        SNAPSHOT_JSON = os.path.join(RUN_DIR, f"maze_{MAZE_SIZE}x{MAZE_SIZE}.json")
        FIG_DIR = os.path.join(RUN_DIR, 'figs')
        LOG_DIR = os.path.join(RUN_DIR, 'logs')
        MPL_DIR = os.path.join(RUN_DIR, 'mpl')
        os.makedirs(FIG_DIR, exist_ok=True)
        print('SUMMARY_PATH not found; using latest run:', RUN_DIR)
        print('SUMMARY_PATH:', SUMMARY_PATH)
    else:
        raise FileNotFoundError(
            f"No summary found under {DRIVE_ROOT}. Re-run the experiment cell first."
        )
else:
    print('Using SUMMARY_PATH:', SUMMARY_PATH)


## 6. Inspect Results


In [16]:
!ls -lh "{RUN_DIR}" | tail -n 20


total 12K
drwx------ 2 root root 4.0K Dec 29 15:27 figs
drwx------ 2 root root 4.0K Dec 29 15:27 logs
drwx------ 2 root root 4.0K Dec 29 15:27 mpl


## 7. Export Metrics (Paper-Ready)


In [17]:
!python experiments/maze-query-hub-prototype/tools/export_paper_maze.py   --summary "{SUMMARY_PATH}"   --steps "{STEPS_PATH}"   --out-json "{RUN_DIR}/maze_{MAZE_SIZE}x{MAZE_SIZE}_metrics.json"   --out-csv "{RUN_DIR}/maze_{MAZE_SIZE}x{MAZE_SIZE}_metrics.csv"   --compression-base mem


Traceback (most recent call last):
  File "/content/InsightSpike-AI/experiments/maze-query-hub-prototype/tools/export_paper_maze.py", line 155, in <module>
    main()
  File "/content/InsightSpike-AI/experiments/maze-query-hub-prototype/tools/export_paper_maze.py", line 138, in main
    summary = load_json(args.summary)
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/InsightSpike-AI/experiments/maze-query-hub-prototype/tools/export_paper_maze.py", line 29, in load_json
    return json.loads(path.read_text(encoding="utf-8"))
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1027, in read_text
    with self.open(mode='r', encoding=encoding, errors=errors) as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1013, in open
    return io.open(self, mode, buffering, encoding, errors, newline)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundE

## 8. Final Visualizations


In [18]:
import json
import os
import matplotlib.pyplot as plt

SNAPSHOT_STEP = 200

with open(SUMMARY_PATH, 'r') as f:
    summary = json.load(f)

runs = summary.get('runs', [])
if not runs:
    raise ValueError('No runs found in summary.')

run = runs[0]
seed = run.get('seed', SEED_START)

g0 = run.get('g0_series', [])
gmin = run.get('gmin_series', [])

plt.figure(figsize=(8, 3.5))
plt.plot(g0, label='g0')
plt.plot(gmin, label='gmin')
plt.xlabel('step')
plt.ylabel('g')
plt.title(f'g0/gmin time series (seed={seed})')
plt.grid(True, linewidth=0.3, alpha=0.5)
plt.legend()

g_series_path = os.path.join(FIG_DIR, f'g0_gmin_{MAZE_SIZE}x{MAZE_SIZE}_seed{seed}.png')
plt.savefig(g_series_path, dpi=180, bbox_inches='tight')
plt.show()
print('Saved:', g_series_path)

with open(STEPS_PATH, 'r') as f:
    steps = json.load(f)

step_item = next((s for s in steps if s.get('seed') == seed and s.get('step') == SNAPSHOT_STEP), None)
if step_item is None:
    candidates = [s for s in steps if s.get('seed') == seed]
    if not candidates:
        raise ValueError('No step logs found for the selected seed.')
    idx = min(SNAPSHOT_STEP, len(candidates) - 1)
    step_item = candidates[idx]

nodes = step_item.get('graph_nodes', [])
if not nodes:
    raise ValueError('No graph_nodes found in step log.')

rows = [n[0] for n in nodes]
cols = [n[1] for n in nodes]

plt.figure(figsize=(6, 6))
plt.scatter(cols, rows, s=8, alpha=0.85)
plt.gca().invert_yaxis()
plt.xlabel('col')
plt.ylabel('row')
plt.title(f'Graph growth snapshot ({MAZE_SIZE}x{MAZE_SIZE}, step={step_item.get("step")})')
plt.grid(True, linewidth=0.3, alpha=0.4)

snap_path = os.path.join(FIG_DIR, f'graph_snapshot_{MAZE_SIZE}x{MAZE_SIZE}_step{step_item.get("step")}.png')
plt.savefig(snap_path, dpi=180, bbox_inches='tight')
plt.show()
print('Saved:', snap_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/insightspike/maze_51x51_full/maze51x51_s1500_seeds13_20251229_152707/_51x51_s1500_l3only_summary.json'